In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('kepler_confirmed.csv')


In [3]:
def classify_planet(planet):
    # Extract radius and mass from the planet data
    planet_radius_earth = planet.get('pl_rade')  # in Earth radii
    planet_radius_jupiter = planet.get('pl_radj')  # in Jupiter radii
    planet_mass_earth = planet.get('pl_bmasse')  # in Earth masses
    planet_mass_jupiter = planet.get('pl_bmassj')  # in Jupiter masses

    # If only Jupiter-radius or Jupiter-mass is provided, convert to Earth equivalents
    if planet_radius_jupiter:
        planet_radius_earth = planet_radius_jupiter * 11.21  # 1 Jupiter radius = 11.21 Earth radii
    if planet_mass_jupiter:
        planet_mass_earth = planet_mass_jupiter * 317.8  # 1 Jupiter mass = 317.8 Earth masses

    # Classification based on radius and mass
    if planet_radius_earth:
        if planet_radius_earth <= 1.5:
            planet_type = 'Terrestrial (rocky planet like Earth)'
        elif 1.5 < planet_radius_earth <= 2.0:
            planet_type = 'Super-Earth (larger rocky planet)'
        elif 2.0 < planet_radius_earth <= 4:
            if planet_mass_earth and planet_mass_earth > 10:
                planet_type = 'Mostly Gas (gas-dominant planet like Neptune)'
            else:
                planet_type = 'Neptune-like (gas planet like Neptune)'
        elif planet_radius_earth > 4 and planet_radius_earth <= 10:
            if planet_mass_earth and planet_mass_earth > 17:
                planet_type = 'Mostly Gas (large gas-dominant planet)'
            else:
                planet_type = 'Neptune-like (large gas-dominant planet)'
        elif planet_radius_earth > 10:
            planet_type = 'Jupiter-like (gas giant like Jupiter)'
        else:
            planet_type = 'Unknown type (in between sizes)'
    elif planet_mass_earth:
        if planet_mass_earth <= 1.5:
            planet_type = 'Terrestrial (based on mass)'
        elif 1.5 < planet_mass_earth <= 10:
            planet_type = 'Super-Earth (based on mass)'
        elif 10 < planet_mass_earth <= 17:
            planet_type = 'Neptune-like (based on mass)'
        elif planet_mass_earth > 17 and planet_mass_earth <= 100:
            planet_type = 'Mostly Gas (gas-dominant planet)'
        elif planet_mass_earth > 100:
            planet_type = 'Jupiter-like (gas giant like Jupiter)'
        else:
            planet_type = 'Unknown type (in between masses)'
    else:
        planet_type = 'Unknown type (insufficient data)'

    return planet_type


In [4]:
df['planet_classification'] = df.apply(classify_planet, axis=1)


In [5]:
def approximate_color_hex(planet):
    """
    Approximate the color of an exoplanet based on its equilibrium temperature
    and the effective temperature of its host star, returning hexadecimal color values.
    
    Parameters:
    - planet (dict): A dictionary containing exoplanet data.

    Returns:
    - str: Estimated color of the exoplanet in hexadecimal format.
    """
    # Extract relevant data
    pl_eqt = planet.get('pl_eqt')  # Equilibrium temperature in Kelvin
    st_teff = planet.get('st_teff')  # Star's effective temperature in Kelvin

    # Initialize color based on temperature with vibrant colors
    if pl_eqt is not None:
        if pl_eqt < 300:
            return '#ADD8E6'  # Light Blue
        elif 300 <= pl_eqt < 600:
            return '#90EE90'  # Light Green
        elif 600 <= pl_eqt < 900:
            return '#FFD700'  # Gold (Warm Yellow)
        elif 900 <= pl_eqt < 1200:
            return '#FFA500'  # Orange
        else:
            return '#FF4500'  # Orange Red
    elif st_teff is not None:
        if st_teff < 4000:
            return '#FF6347'  # Tomato (Red)
        elif 4000 <= st_teff < 5700:
            return '#FFD700'  # Gold (Yellow)
        elif 5700 <= st_teff < 7500:
            return '#FFFFFF'  # White
        else:
            return '#ADD8E6'  # Light Blue
    
    return '#808080'  

In [6]:
df['planet_colour_approximation'] = df.apply(approximate_color_hex, axis=1)

In [7]:
df.to_json('kepler_confirmed.json', orient='records')